In [1]:
# import libraries

import ee
import geemap
from datetime import datetime
import time
import pandas as pd

In [2]:
# authorize access and initial ee client

ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AZEOvhWlnhtirgP0_9U9R3MOlOnmg3P_2XJRqg24ZTb_xBPsOv0H-reQq4Q

Successfully saved authorization token.


In [3]:
# define region of interest

roi = ee.Geometry.Rectangle([-160, 0, 0, 50])

In [4]:
# get CO data from Sentinel-5

d1 = '2023-06-27'
d2 = '2023-07-02'

co_collection = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_CO').select('CO_column_number_density').filterDate(d1, d2);

n = co_collection.size().getInfo()
print('Number of images:', n)

Number of images: 780


In [35]:
# get dates of images

acq_times = co_collection.aggregate_array('system:time_start').getInfo()
date_list = [time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times]
date_list

['06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-2023',
 '06-01-20

In [5]:
# define visualization parameters and show data on map

co_vis = {
  'min': 0,
  'max': 0.05,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
  'opacity': 0.5
};

Map = geemap.Map()
Map.centerObject(roi, 4)
Map.addLayer(co_collection.median().clip(roi), co_vis, 'co')
Map

Map(center=[40.15601627364505, -80], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…

In [6]:
# image get resolution

n = co_collection.size().getInfo()
co_collection_list = co_collection.toList(n)
ns = ee.Image(co_collection_list.get(0)).projection().nominalScale().getInfo()
print(ns)

1113.1949079327358


In [11]:
print(ns*10)

11131.949079327358


In [13]:
# export daily CO mean 

#d = pd.date_range(start='2023-06-01',end='2023-06-28')
d = pd.date_range(start='2023-05-30',end='2023-07-02')
n = co_collection.size().getInfo()
co_collection_list = co_collection.toList(n)
ns = ee.Image(co_collection_list.get(0)).projection().nominalScale().getInfo()
task_id = []

for i in range(len(d)-1):
    d1 = d[i].strftime('%Y-%m-%d')
    d2 = d[i+1].strftime('%Y-%m-%d')
    
    co_collection = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_CO').select('CO_column_number_density').filterDate(d1, d2);
    co_img = co_collection.mean().clip(roi);

    txt = 's5_co_0p1deg_' + d1
    task = ee.batch.Export.image.toDrive(image=co_img,
                                         description=txt,
                                         scale=ns*10,
                                         region=roi,
                                         fileNamePrefix=txt,
                                         crs='EPSG:4326',
                                         fileFormat='GeoTIFF',
                                         maxPixels=2e10)
    task.start()
    task_id.append(task.id)
    print(d1)



2023-05-30
2023-05-31
2023-06-01
2023-06-02
2023-06-03
2023-06-04
2023-06-05
2023-06-06
2023-06-07
2023-06-08
2023-06-09
2023-06-10
2023-06-11
2023-06-12
2023-06-13
2023-06-14
2023-06-15
2023-06-16
2023-06-17
2023-06-18
2023-06-19
2023-06-20
2023-06-21
2023-06-22
2023-06-23
2023-06-24
2023-06-25
2023-06-26
2023-06-27
2023-06-28
2023-06-29
2023-06-30
2023-07-01


In [15]:
# print status of each task

task.list()

[<Task QELYAR3HEY77RFB3K2KNJBNA EXPORT_IMAGE: s5_co_0p1deg_2023-07-01 (READY)>,
 <Task H3HLF3QQPAEEYVXWO55KILEZ EXPORT_IMAGE: s5_co_0p1deg_2023-06-30 (READY)>,
 <Task T6X7UKSGO67RRHZ5LUJLQTGI EXPORT_IMAGE: s5_co_0p1deg_2023-06-29 (READY)>,
 <Task 7J2ZRQZA3HOL56LLDMKHK5TN EXPORT_IMAGE: s5_co_0p1deg_2023-06-28 (READY)>,
 <Task ANKMFEGRWM66SEQF4V4CNFFF EXPORT_IMAGE: s5_co_0p1deg_2023-06-27 (READY)>,
 <Task 2FBTZZUEKZ6ALUZ5BJDVOM4G EXPORT_IMAGE: s5_co_0p1deg_2023-06-26 (READY)>,
 <Task QYGPUONQU6L5W475H55I3WUH EXPORT_IMAGE: s5_co_0p1deg_2023-06-25 (READY)>,
 <Task T7GYSG6Q7JDJVHTFDHFIW7RO EXPORT_IMAGE: s5_co_0p1deg_2023-06-24 (READY)>,
 <Task 27BEYRV2H4LQKX72WJOHA5SL EXPORT_IMAGE: s5_co_0p1deg_2023-06-23 (READY)>,
 <Task SFSOKFKY3GYHIASFCZ7OAHBO EXPORT_IMAGE: s5_co_0p1deg_2023-06-22 (READY)>,
 <Task LLKV3WXGD467B22DPJUD26BG EXPORT_IMAGE: s5_co_0p1deg_2023-06-21 (READY)>,
 <Task JXMOE3DB7TNBOSY7CMUWNN4B EXPORT_IMAGE: s5_co_0p1deg_2023-06-20 (READY)>,
 <Task 6I6FIT5UTEQ3BTIE4JYJ6QEP EXPORT_I

In [63]:
#task.status()
t_id = "3S2GCUOTXUENRD6KP36B2VJM"
print(ee.data.getTaskStatus(t_id))

[{'state': 'FAILED', 'description': 'srtm_90m_us_and_canada', 'creation_timestamp_ms': 1688073547537, 'update_timestamp_ms': 1688076291480, 'start_timestamp_ms': 1688076289676, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': 'Export too large: specified 12344376736 pixels (max: 2000000000). Specify higher maxPixels value if you intend to export a large area.', 'id': '3S2GCUOTXUENRD6KP36B2VJM', 'name': 'projects/earthengine-legacy/operations/3S2GCUOTXUENRD6KP36B2VJM'}]
